In [4]:
# pip install --upgrade xee
import os
import numpy as np
import geopandas as gpd
import pandas as pd

# import sliderule
# from sliderule import icesat2, gedi, earthdata

import matplotlib.pyplot as plt
import seaborn as sns

import xarray as xr

import ee
import geemap
# ee.Authenticate()
# Initialize the Earth Engine API
ee.Initialize()

ModuleNotFoundError: No module named 'geemap'

In [ ]:
#####################
## Read roi
roi_path = '/projects/my-private-bucket/HLS-1DCNN-AGB/data/shp/atlantic_forest/AtlanticForest.shp' 
roi_gdf = gpd.read_file(tiles_path)



In [ ]:
# Dissolve grid and make a 60 m buffer
roi_gdf = roi_gdf.dissolve()
roi_gdf = roi_gdf.buffer(60)
roi_gdf = gpd.GeoDataFrame(geometry=roi_gdf, crs=roi_gdf.crs)

In [ ]:
# roi_gdf.plot()

In [2]:
roi_gdf = roi_gdf.to_crs('EPSG:4326')


NameError: name 'roi_gdf' is not defined

In [3]:
roi_fc = geemap.geopandas_to_ee(roi_gdf).first().geometry()   


NameError: name 'geemap' is not defined

In [80]:
# Vegetation mask
# https://brasil.mapbiomas.org/en/colecoes-mapbiomas/
mapbiomas = ee.Image('projects/mapbiomas-public/assets/brazil/lulc/collection10/mapbiomas_brazil_collection10_coverage_v2')
mapbiomas = mapbiomas.clip(roi_fc)


In [96]:
# Filter Mapbiomas for specifc year, set the vegetation ids
# years = [2019,2020,2021,2022]
year = 2020
classification_year =  'classification_' + str(year)

vegetation_ids = ee.List([1, 3, 4, 5, 6, 49])

# Select year and vegetation id
mapbiomas_roi = mapbiomas.select([classification_year]).clip(roi_fc)

# Create mask only with pixels with vegetation classified
mapbiomas_roi_veg = mapbiomas_roi.updateMask(
    mapbiomas_roi.eq(
        ee.Image.constant(vegetation_ids)).reduce(
            ee.Reducer.anyNonZero()))

In [97]:
# Visualize
veg_binary = mapbiomas_roi_veg.gt(0).selfMask().rename('veg')

m = geemap.Map()
m.centerObject(roi_fc, 6)             # adjust zoom as you like
m.addLayer(veg_binary, {'min': 0, 'max': 1, 'palette': ['#2E8B57']}, 'Vegetation mask')
m.addLayer(roi_fc, {}, 'ROI', False)

m

Map(center=[-20.651744248759403, -46.054006218425144], controls=(WidgetControl(options=['position', 'transpare…

In [98]:
## Define grid to download tiles
fishnet_aoi = geemap.fishnet(roi_fc, rows=10, cols=20)

In [99]:
out_dir = f'/projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/VegetationMask/AtlanticForest/{year}/Tiles'
os.makedirs(out_dir,exist_ok= True)

In [100]:

geemap.download_ee_image_tiles(
    mapbiomas_roi_veg, fishnet_aoi, out_dir, prefix=f"vegmask{year}_", crs="EPSG:4326", scale=30
)

...iomas_brazil_collection10_coverage_v2:   0%|          |0/19 tiles [00:00<?]

...iomas_brazil_collection10_coverage_v2:   0%|          |0/19 tiles [00:00<?]

...iomas_brazil_collection10_coverage_v2:   0%|          |0/19 tiles [00:00<?]

...iomas_brazil_collection10_coverage_v2:   0%|          |0/19 tiles [00:00<?]

...iomas_brazil_collection10_coverage_v2:   0%|          |0/19 tiles [00:00<?]

...

...iomas_brazil_collection10_coverage_v2:   0%|          |0/19 tiles [00:00<?]

...iomas_brazil_collection10_coverage_v2:   0%|          |0/19 tiles [00:00<?]

...iomas_brazil_collection10_coverage_v2:   0%|          |0/19 tiles [00:00<?]

...iomas_brazil_collection10_coverage_v2:   0%|          |0/19 tiles [00:00<?]

Downloaded 86 tiles in 651.3073766231537 seconds.


## Create VRTs

In [ ]:
# gdalbuildvrt \
#   -srcnodata 0 -vrtnodata 0 \
#   /projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/VegetationMask/AtlanticForest/2024/vegmask_2024.vrt \
#   /projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/VegetationMask/AtlanticForest/2024/Tiles/*.tif